In [40]:
import re
import nltk
from nltk.corpus import reuters
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer    
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
import warnings
warnings.filterwarnings('ignore')
#TfidfVectorizer :빈도로 나온다.

In [41]:
import nltk
nltk.download('reuters')
nltk.download('all')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\DKU\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [2]:
from nltk.corpus import reuters
#문서 확인
reuters.fileids()

['test/14826',
 'test/14828',
 'test/14829',
 'test/14832',
 'test/14833',
 'test/14839',
 'test/14840',
 'test/14841',
 'test/14842',
 'test/14843',
 'test/14844',
 'test/14849',
 'test/14852',
 'test/14854',
 'test/14858',
 'test/14859',
 'test/14860',
 'test/14861',
 'test/14862',
 'test/14863',
 'test/14865',
 'test/14867',
 'test/14872',
 'test/14873',
 'test/14875',
 'test/14876',
 'test/14877',
 'test/14881',
 'test/14882',
 'test/14885',
 'test/14886',
 'test/14888',
 'test/14890',
 'test/14891',
 'test/14892',
 'test/14899',
 'test/14900',
 'test/14903',
 'test/14904',
 'test/14907',
 'test/14909',
 'test/14911',
 'test/14912',
 'test/14913',
 'test/14918',
 'test/14919',
 'test/14921',
 'test/14922',
 'test/14923',
 'test/14926',
 'test/14928',
 'test/14930',
 'test/14931',
 'test/14932',
 'test/14933',
 'test/14934',
 'test/14941',
 'test/14943',
 'test/14949',
 'test/14951',
 'test/14954',
 'test/14957',
 'test/14958',
 'test/14959',
 'test/14960',
 'test/14962',
 'test/149

In [44]:
categories = reuters.categories() #카테고리 개수 확인
print("Reuters has %d categories:\n%s" % (len(categories), categories))

Reuters has 90 categories:
['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


In [61]:
#각 카테고리에 대한 documents 개수 확인 
total = len(reuters.paras())
df=pd.DataFrame()

for i in categories:
    real = len(reuters.paras(categories=[i]))
    df=te.append(pd.DataFrame([[i,real,round(100*(real/total),1)]],columns=['i','real','percent']),ignore_index=True)

print(df.sort_values(by=['percent'],ascending=False))
#불균형 데이터임을 알 수 있다.

             i  real  percent
21        earn  4211     35.4
0          acq  2661     22.4
46    money-fx   835      7.0
17       crude   658      5.5
26       grain   640      5.4
..         ...   ...      ...
58  palmkernel     3      0.0
18         dfl     3      0.0
70         rye     3      0.0
63        rand     3      0.0
37         jet     5      0.0

[91 rows x 3 columns]


In [5]:
paras=nltk.corpus.reuters.paras()
paras#문단으로 끊은 것

[[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ['But', 'some', 'exporters', 'said', 'that', 'while', 'the', 'conflict', 'would', 'hurt', 'them', 'in', 'the', 'long', '-', 'run', ',', 'in', 'the', 'short', '-', 'term', 'Tokyo', "'", 's', 'loss', 'might', 'be', 'their', 'gain', '.'], ['The', 'U', '.', 'S', '.', 'Has', 'said', 'it', 'will', 

#map은 연산 filter는 true, false에 대한 값 출력 한번사용할거면 간결하게 lambda이용, 아니면 함수 def로 만든다.

In [6]:
documents=reuters.fileids()
train_doc=list(filter(lambda doc: doc.startswith("train"),documents))
test_doc=list(filter(lambda doc: doc.startswith("test"),documents))
train_documents, train_categories = zip(*[(reuters.raw(i), reuters.categories(i)) for i in reuters.fileids() if i.startswith('training/')])
test_documents, test_categories = zip(*[(reuters.raw(i), reuters.categories(i)) for i in reuters.fileids() if i.startswith('test/')])

In [7]:
train_documents[1]

"COMPUTER TERMINAL SYSTEMS &lt;CPML> COMPLETES SALE\n  Computer Terminal Systems Inc said\n  it has completed the sale of 200,000 shares of its common\n  stock, and warrants to acquire an additional one mln shares, to\n  &lt;Sedio N.V.> of Lugano, Switzerland for 50,000 dlrs.\n      The company said the warrants are exercisable for five\n  years at a purchase price of .125 dlrs per share.\n      Computer Terminal said Sedio also has the right to buy\n  additional shares and increase its total holdings up to 40 pct\n  of the Computer Terminal's outstanding common stock under\n  certain circumstances involving change of control at the\n  company.\n      The company said if the conditions occur the warrants would\n  be exercisable at a price equal to 75 pct of its common stock's\n  market price at the time, not to exceed 1.50 dlrs per share.\n      Computer Terminal also said it sold the technolgy rights to\n  its Dot Matrix impact technology, including any future\n  improvements, to &lt;

In [8]:
# 문장부호 삭제
import re

#대소문자 변경, 문장부호 삭제
def cleanText(text):
    text = text.lower()
    # replace line breaks with one space
    text = re.sub(r"\n", "", text)
    # replace multiple spacess with one space
    text = re.sub('\s+', ' ', text)  
    # 특수문자 삭제
    text=re.sub(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^_\`{|}~\\\\]','',text)
    # replace '\' by nothing because there are a lot in the text for some reason
    text.replace("\\","")
    return text

In [9]:
#확인용
text1=cleanText(train_documents[1])
text1

'computer terminal systems ltcpml completes sale computer terminal systems inc said it has completed the sale of 200000 shares of its common stock and warrants to acquire an additional one mln shares to ltsedio nv of lugano switzerland for 50000 dlrs the company said the warrants are exercisable for five years at a purchase price of 125 dlrs per share computer terminal said sedio also has the right to buy additional shares and increase its total holdings up to 40 pct of the computer terminals outstanding common stock under certain circumstances involving change of control at the company the company said if the conditions occur the warrants would be exercisable at a price equal to 75 pct of its common stocks market price at the time not to exceed 150 dlrs per share computer terminal also said it sold the technolgy rights to its dot matrix impact technology including any future improvements to ltwoodco inc of houston tex for 200000 dlrs but it said it would continue to be the exclusive w

In [35]:
#실제 data 처리, #token처리하려면 text뒤에 .split() 붙여준다.
train_split= []
for document in train_documents:
    text = cleanText(document)
    train_split.append(text)

In [32]:
train_split

['bahia cocoa review showers continued throughout the week in the bahia cocoa zone alleviating the drought since early january and improving prospects for the coming temporao although normal humidity levels have not been restored comissaria smith said in its weekly review the dry period means the temporao will be late this year arrivals for the week ended february 22 were 155221 bags of 60 kilos making a cumulative total for the season of 593 mln against 581 at the same stage last year again it seems that cocoa delivered earlier on consignment was included in the arrivals figures comissaria smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end with total bahia crop estimates around 64 mln bags and sales standing at almost 62 mln there are a few hundred thousand bags still in the hands of farmers middlemen exporters and processors there are doubts as to how much of this cocoa would be fit for export as shippers

In [36]:
## Getting a string of the full corpus
fullText = ""
for document in train_documents: 
    text = cleanText(document)
    fullText = fullText + text
type(fullText)

str

In [13]:
stopWords = stopwords.words('english')
charfilter = re.compile('[a-zA-Z]+')

def simple_tokenizer(text):
    #tokenizing the words
    words = word_tokenize(text)
    
    #remove every stopwords
    words = [word for word in words if word not in stopWords]
    
    #lemmatization all the tokens 
    tokens = (list(map(lambda token:WordNetLemmatizer().lemmatize(token), words)))
    #dtm에서 숫자또한 빈도로 하지 않게 
    tokens = list(filter(lambda token : charfilter.match(token),tokens))
    return tokens

#DTM 만들기 (vectorize the data using Countvector:1%미만 제거)
count = CountVectorizer(tokenizer = simple_tokenizer,min_df=0.01)
train_count=count.fit_transform(train_split)

df_dtm = pd.DataFrame(train_count.toarray(), index=train_doc, columns=count.get_feature_names())
df_dtm

,able,accept,accepted,accord,according,account,accounting,acquire,acquired,acquisition,...,world,worth,would,year,yearago,yen,yesterday,yet,yield,york
training/1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,0,0,0,0,8
training/10,0,0,0,0,0,0,0,1,0,0,...,0,0,3,1,0,0,0,0,0,0
training/100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
training/1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
training/10000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
training/999,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
training/9992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
training/9993,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
training/9994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## TF-IDF
DF: dtm(TF)에서 각 컬럼에 0이 아닌 횟수
IDF: log(n/(1+df)) n:문서 수 
TF-IDF: TF*IDF 단 0을 곱하면 0이기에 단어가 하나라도 있으면 최솟값인 1을 사용한다.

In [20]:
tfidf = TfidfVectorizer(tokenizer = simple_tokenizer,min_df=0.01)
train_tfidf=tfidf.fit_transform(train_split)

df_tfidf = pd.DataFrame(train_tfidf.toarray(), index=train_doc, columns=tfidf.get_feature_names())
df_tfidf

,able,accept,accepted,accord,according,account,accounting,acquire,acquired,acquisition,...,world,worth,would,year,yearago,yen,yesterday,yet,yield,york
training/1,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.029875,0.045614,0.0000,0.0,0.00000,0.0,0.0,0.405249
training/10,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.074226,0.0,0.0,...,0.0,0.0,0.138999,0.035372,0.0000,0.0,0.00000,0.0,0.0,0.000000
training/100,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.097640,0.0000,0.0,0.00000,0.0,0.0,0.000000
training/1000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.10744,0.0,0.0,0.000000
training/10000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.066207,0.1814,0.0,0.00000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
training/999,0.0,0.0,0.0,0.0,0.0,0.17321,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.00000,0.0,0.0,0.000000
training/9992,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.00000,0.0,0.0,0.000000
training/9993,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.00000,0.0,0.0,0.000000
training/9994,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.055483,0.0000,0.0,0.00000,0.0,0.0,0.000000


In [ ]:
#FreDist :문서의 사용된 단어의 사용빈도를 담는 클래스
from nltk import FreqDist


In [178]:
count.vocabulary_
##빈도수 많은 10개 출력 밑에 함수 참고해서 만들어보자 다음에~~
most_freq = sorted(word_counter.items(), key=lambda item: item[1], reverse=True)
print(most_freq[:10])

{'review': 714,
 'continued': 176,
 'week': 889,
 'since': 773,
 'early': 249,
 'january': 423,
 'prospect': 647,
 'coming': 144,
 'although': 40,
 'normal': 536,
 'level': 450,
 'said': 725,
 'weekly': 890,
 'period': 597,
 'mean': 493,
 'late': 436,
 'year': 905,
 'ended': 264,
 'february': 306,
 'making': 481,
 'total': 848,
 'season': 732,
 'mln': 507,
 'last': 435,
 'earlier': 248,
 'included': 393,
 'figure': 311,
 'still': 802,
 'much': 516,
 'crop': 193,
 'available': 65,
 'come': 143,
 'end': 263,
 'estimate': 275,
 'around': 55,
 'sale': 726,
 'almost': 36,
 'hand': 368,
 'farmer': 304,
 'exporter': 293,
 'would': 904,
 'export': 292,
 'certificate': 124,
 'view': 881,
 'lower': 471,
 'recent': 676,
 'sold': 778,
 'good': 357,
 'part': 580,
 'held': 372,
 'price': 626,
 'rose': 722,
 'per': 594,
 'offer': 544,
 'shipment': 759,
 'limited': 456,
 'march': 487,
 'dlrs': 241,
 'tonne': 845,
 'port': 608,
 'new': 529,
 'also': 38,
 'light': 452,
 'open': 554,
 'going': 355,
 'yor

## 토큰화
문맥적인 의미는 무시될 수밖에 없다. 입력을 str or bytes-like object 형식을 받고 출력은 list로 반환.
방법 
1. text.split() 예시) here's -> 'here's'
2. NLTK 패키지 쓰기 예시) here's -> 'here',''','s'
3. DTM할 때 CountVectorizer, TfidfVectorizer 이용한다. (소문자 일괄 변환, 토큰화, 스톱워드 필터링 등 전처리 함께 수행한다.) 수정하고 싶으면 앞에 함수 만들고 'tokenizer= '로 입력 한다.

In [37]:
from nltk.tokenize import word_tokenize, sent_tokenize #RegexpTokenizer
def tokenize_text(text):
    sentences=sent_tokenize(text)
    word_tokens=[word_tokenize(sentence) for sentence in sentences]
    return word_tokens
word_tokens=tokenize_text(fullText)


#retokenize = RegexpTokenizer("[\w]+")
#word=retokenize.tokenize(fullText)


In [40]:
#원래 문자로 복구하기

#토큰화된 단어 연결
pharase=[" ".join(i) for i in word_tokens]
#문장 연결
doc='\n'.join(pharase)
#문장 간 연결 부드럽게
doc=doc.replace(",",".").replace('\n','')

'computer terminal systems ltcpml completes sale computer terminal systems inc said it has completed the sale of 200000 shares of its common stock and warrants to acquire an additional one mln shares to ltsedio nv of lugano switzerland for 50000 dlrs the company said the warrants are exercisable for five years at a purchase price of 125 dlrs per share computer terminal said sedio also has the right to buy additional shares and increase its total holdings up to 40 pct of the computer terminals outstanding common stock under certain circumstances involving change of control at the company the company said if the conditions occur the warrants would be exercisable at a price equal to 75 pct of its common stocks market price at the time not to exceed 150 dlrs per share computer terminal also said it sold the technolgy rights to its dot matrix impact technology including any future improvements to ltwoodco inc of houston tex for 200000 dlrs but it said it would continue to be the exclusive w

## 불용어 삭제
분석에 큰 의미가 없는 단어 삭제, str or object입력하면 영어 한 알파벳씩 결과가 나온다.
입력에 list 넣거나 tuple 넣으면 깔끔하게 결과 list로 나온다.

In [185]:
type(train_splitstop)

list

In [186]:
from nltk.corpus import stopwords
def content_fraction(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [i for i in text if i not in stopwords]
    return content
train_splitstop=content_fraction(train_split)
train_splitstop

['bahia cocoa review showers continued throughout the week in the bahia cocoa zone alleviating the drought since early january and improving prospects for the coming temporao although normal humidity levels have not been restored comissaria smith said in its weekly review the dry period means the temporao will be late this year arrivals for the week ended february 22 were 155221 bags of 60 kilos making a cumulative total for the season of 593 mln against 581 at the same stage last year again it seems that cocoa delivered earlier on consignment was included in the arrivals figures comissaria smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end with total bahia crop estimates around 64 mln bags and sales standing at almost 62 mln there are a few hundred thousand bags still in the hands of farmers middlemen exporters and processors there are doubts as to how much of this cocoa would be fit for export as shippers

In [56]:
#불용어 삭제한 후 얼마나 삭제됐나 확인
from nltk.corpus import stopwords
def content_fraction(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [i for i in text if i not in stopwords]
    return len(content) / len(text)

content_fraction(train_split[0])
#감소한 것을 알 수 있다.

0.6605316973415133

## 어근 동일화(어간추출) (stemming)
문법적으로 변화하는 단어의 원형을 찾는 것
한국어인 경우 라이브러리가 없어 형태소 분석기 이용한다.
단순히 어미를 제거할 뿐이므로 단어의 원형을 정확히 찾아주지는 않는다. 따라서 본 분석에서는 원형복원만 실행했다.

In [196]:
from nltk.stem.porter import PorterStemmer #LancasterStemmer, RegexpStemmer

stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for i in tokens:
        stemmed.append(stemmer.stem(i))
    return stemmed
train_split_stem=stem_tokens(train_split,stemmer)
train_split_stem

['bahia cocoa review showers continued throughout the week in the bahia cocoa zone alleviating the drought since early january and improving prospects for the coming temporao although normal humidity levels have not been restored comissaria smith said in its weekly review the dry period means the temporao will be late this year arrivals for the week ended february 22 were 155221 bags of 60 kilos making a cumulative total for the season of 593 mln against 581 at the same stage last year again it seems that cocoa delivered earlier on consignment was included in the arrivals figures comissaria smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end with total bahia crop estimates around 64 mln bags and sales standing at almost 62 mln there are a few hundred thousand bags still in the hands of farmers middlemen exporters and processors there are doubts as to how much of this cocoa would be fit for export as shippers

## 원형복원
의미적으로 변화하는 단어의 원형을 찾는 것

In [197]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens, lemmatizer):
    lemmatized = []
    for i in tokens:
        lemmatized.append(lemmatizer.lemmatize(i))
    return lemmatized
lemmatize_tokens(train_split,lemmatizer)

['bahia cocoa review showers continued throughout the week in the bahia cocoa zone alleviating the drought since early january and improving prospects for the coming temporao although normal humidity levels have not been restored comissaria smith said in its weekly review the dry period means the temporao will be late this year arrivals for the week ended february 22 were 155221 bags of 60 kilos making a cumulative total for the season of 593 mln against 581 at the same stage last year again it seems that cocoa delivered earlier on consignment was included in the arrivals figures comissaria smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end with total bahia crop estimates around 64 mln bags and sales standing at almost 62 mln there are a few hundred thousand bags still in the hands of farmers middlemen exporters and processors there are doubts as to how much of this cocoa would be fit for export as shippers

In [22]:
#품사부착(POS) -이런것도 있다~
from nltk.tag import pos_tag
tagged_list = pos_tag(word_tokenize(text1)) #토큰으로 나눈 문장 이용

#명사인 토큰만 확인 가능
nouns_list = [t[0] for t in tagged_list if t[1] == "NN"]

#Scikit-Learn 등에서 자연어 분석을 할 때는 같은 토큰이라도 품사가 다르면 다른 토큰으로 처리해야 하는 경우가 많은데 이 때는 원래의 토큰과 품사를 붙여서 새로운 토큰 이름을 만들어 사용하면 철자가 같고 품사가 다른 단어를 구분할 수 있다.
def tokenizer(doc):
    return ["/".join(p) for p in tagged_list]

tokenizer(text1)

['computer/NN',
 'terminal/JJ',
 'systems/NNS',
 'ltcpml/VBP',
 'completes/NNS',
 'sale/NN',
 'computer/NN',
 'terminal/JJ',
 'systems/NNS',
 'inc/NN',
 'said/VBD',
 'it/PRP',
 'has/VBZ',
 'completed/VBN',
 'the/DT',
 'sale/NN',
 'of/IN',
 '200000/CD',
 'shares/NNS',
 'of/IN',
 'its/PRP$',
 'common/JJ',
 'stock/NN',
 'and/CC',
 'warrants/NNS',
 'to/TO',
 'acquire/VB',
 'an/DT',
 'additional/JJ',
 'one/CD',
 'mln/NN',
 'shares/NNS',
 'to/TO',
 'ltsedio/VB',
 'nv/NN',
 'of/IN',
 'lugano/NN',
 'switzerland/NN',
 'for/IN',
 '50000/CD',
 'dlrs/NN',
 'the/DT',
 'company/NN',
 'said/VBD',
 'the/DT',
 'warrants/NNS',
 'are/VBP',
 'exercisable/JJ',
 'for/IN',
 'five/CD',
 'years/NNS',
 'at/IN',
 'a/DT',
 'purchase/NN',
 'price/NN',
 'of/IN',
 '125/CD',
 'dlrs/NN',
 'per/IN',
 'share/NN',
 'computer/NN',
 'terminal/NN',
 'said/VBD',
 'sedio/NN',
 'also/RB',
 'has/VBZ',
 'the/DT',
 'right/NN',
 'to/TO',
 'buy/VB',
 'additional/JJ',
 'shares/NNS',
 'and/CC',
 'increase/VB',
 'its/PRP$',
 'total/JJ

In [ ]:
# 비중 높은 10개 주제 정리

#article별로 단어 빈도 추출

#주제 및 단어별로 2차원 분할표작성